### Описание проекта
В нашем распоряжении есть данные компании «Мегалайн» — федерального оператора сотовой связи. Клиентам предлагают два тарифных плана: «Смарт» и «Ультра». Чтобы скорректировать рекламный бюджет, коммерческий департамент хочет понять, какой тариф приносит больше денег.

Сделаем предварительный анализ тарифов на небольшой выборке клиентов. У нас имеются данные 500 пользователей «Мегалайна»: кто они, откуда, каким тарифом пользуются, сколько звонков и сообщений каждый отправил за 2018 год. Нужно проанализировать поведение клиентов и сделать вывод — какой тариф лучше.

### Описание тарифов

#### Тариф «Смарт»
Ежемесячная плата: **550 рублей**

Включено **500 минут** разговора, 

**50 сообщений**, 

**15 Гб интернет-трафика**

*Стоимость услуг сверх тарифного пакета:*

минута разговора: **3 рубля**

сообщение: **3 рубля**

1 Гб интернет-трафика: **200 рублей**

#### Тариф «Ультра»
Ежемесячная плата: **1950 рублей**

Включено **3000 минут** разговора, 

**1000 сообщений**, 

**30 Гб интернет-трафика**

*Стоимость услуг сверх тарифного пакета:*

минута разговора: **1 рубль**

сообщение: **1 рубль**

1 Гб интернет-трафика: **150 рублей**

Обратим внимание: «Мегалайн» всегда округляет вверх значения минут и мегабайтов. Если пользователь проговорил всего 1 секунду, в тарифе засчитывается целая минута.

Считаем все наши данные

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from scipy import stats as st

In [2]:
calls = pd.read_csv('/datasets/calls.csv') 
internet = pd.read_csv('/datasets/internet.csv')
messages = pd.read_csv('/datasets/messages.csv')
tariffs = pd.read_csv('/datasets/tariffs.csv')
users = pd.read_csv('/datasets/users.csv')

In [3]:
calls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202607 entries, 0 to 202606
Data columns (total 4 columns):
id           202607 non-null object
call_date    202607 non-null object
duration     202607 non-null float64
user_id      202607 non-null int64
dtypes: float64(1), int64(1), object(2)
memory usage: 6.2+ MB


In [4]:
internet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149396 entries, 0 to 149395
Data columns (total 5 columns):
Unnamed: 0      149396 non-null int64
id              149396 non-null object
mb_used         149396 non-null float64
session_date    149396 non-null object
user_id         149396 non-null int64
dtypes: float64(1), int64(2), object(2)
memory usage: 5.7+ MB


In [5]:
messages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123036 entries, 0 to 123035
Data columns (total 3 columns):
id              123036 non-null object
message_date    123036 non-null object
user_id         123036 non-null int64
dtypes: int64(1), object(2)
memory usage: 2.8+ MB


In [6]:
tariffs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 8 columns):
messages_included        2 non-null int64
mb_per_month_included    2 non-null int64
minutes_included         2 non-null int64
rub_monthly_fee          2 non-null int64
rub_per_gb               2 non-null int64
rub_per_message          2 non-null int64
rub_per_minute           2 non-null int64
tariff_name              2 non-null object
dtypes: int64(7), object(1)
memory usage: 256.0+ bytes


In [7]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
user_id       500 non-null int64
age           500 non-null int64
churn_date    38 non-null object
city          500 non-null object
first_name    500 non-null object
last_name     500 non-null object
reg_date      500 non-null object
tariff        500 non-null object
dtypes: int64(2), object(6)
memory usage: 31.4+ KB


### Обрабтка данных

Изучим таблицу `users`

In [8]:
users.head(10)

,user_id,age,churn_date,city,first_name,last_name,reg_date,tariff
0,1000,52,NaN,Краснодар,Рафаил,Верещагин,2018-05-25,ultra
1,1001,41,NaN,Москва,Иван,Ежов,2018-11-01,smart
2,1002,59,NaN,Стерлитамак,Евгений,Абрамович,2018-06-17,smart
3,1003,23,NaN,Москва,Белла,Белякова,2018-08-17,ultra
4,1004,68,NaN,Новокузнецк,Татьяна,Авдеенко,2018-05-14,ultra
5,1005,67,NaN,Набережные Челны,Афанасий,Горлов,2018-01-25,smart
6,1006,21,NaN,Ульяновск,Леонид,Ермолаев,2018-02-26,smart
7,1007,65,NaN,Москва,Юна,Березина,2018-04-19,smart
8,1008,63,NaN,Челябинск,Рустэм,Пономарёв,2018-12-19,smart
9,1009,24,NaN,Пермь,Василиса,Блинова,2018-03-22,smart


In [9]:
users.isnull().sum()

user_id         0
age             0
churn_date    462
city            0
first_name      0
last_name       0
reg_date        0
tariff          0
dtype: int64

Видим, что прпоуски есть только в столбце `churn_date`. Они означают, что тариф до сих пор актуален. Поэтому можем заменить их все на 0

In [10]:
users['churn_date'].fillna(0, inplace=True)

Рассмотрим таблицу `calls`, посмотрим, есть ли в ней пропуски

In [11]:
display(calls.head(10))

,id,call_date,duration,user_id
0,1000_0,2018-07-25,0.00,1000
1,1000_1,2018-08-17,0.00,1000
2,1000_2,2018-06-11,2.85,1000
3,1000_3,2018-09-21,13.80,1000
4,1000_4,2018-12-15,5.18,1000
5,1000_5,2018-11-02,0.00,1000
6,1000_6,2018-10-18,0.00,1000
7,1000_7,2018-08-22,18.31,1000
8,1000_8,2018-09-15,18.44,1000
9,1000_9,2018-08-15,0.00,1000


In [12]:
calls.isnull().sum()

id           0
call_date    0
duration     0
user_id      0
dtype: int64

Пропусков нет, однако у некоторых звонков продолжительность 0.00 минут. Скорее всего, это пропущенные звонки. Или это могут быть входящие звонки. Но судя по таблице, здесь нет отличия между входящими и исходящими звонками.

В любом случае, если длительность звонка 0.00, то такой звонок в счёт абонентской платой не учитывается. 

Округлим значения `duration` до большего целого числа, согласно правилам округления "Мегалайн"
<a id="cell1"></a>

In [13]:
calls['duration'] = np.ceil(calls['duration']).astype('int')

Получим месяц для каждого звонка и добавим его название в столбец `call_month_data`

In [14]:
calls['month'] = pd.to_datetime(calls['call_date']).dt.month # получаем номер месяца

months = {1:'Январь', 2:'Февраль', 3:'Март', 4:'Апрель', 5:'Май', # преобразуем номер месяца в название
         6:'Июнь', 7:'Июль', 8:'Август', 9:'Сентябрь',
         10:'Октябрь', 11:'Ноябрь', 12:'Декабрь'}

calls['month'].replace(months, inplace=True)
calls.head(10)

,id,call_date,duration,user_id,month
0,1000_0,2018-07-25,0,1000,Июль
1,1000_1,2018-08-17,0,1000,Август
2,1000_2,2018-06-11,3,1000,Июнь
3,1000_3,2018-09-21,14,1000,Сентябрь
4,1000_4,2018-12-15,6,1000,Декабрь
5,1000_5,2018-11-02,0,1000,Ноябрь
6,1000_6,2018-10-18,0,1000,Октябрь
7,1000_7,2018-08-22,19,1000,Август
8,1000_8,2018-09-15,19,1000,Сентябрь
9,1000_9,2018-08-15,0,1000,Август


Посчитаем для каждого пользователя количество звонков и количество минут разговора по месяцам

In [15]:
user_calls_by_months = (calls
             .pivot_table(index=['user_id', 'month'], values='duration', aggfunc=['count', 'sum'])
             .reset_index())
user_calls_by_months.columns=['user_id', 'month', 'total_calls', 'total_minutes'] # меняем название столбцов на более логичные
user_calls_by_months.head(10)

,user_id,month,total_calls,total_minutes
0,1000,Август,52,408
1,1000,Декабрь,46,333
2,1000,Июль,47,340
3,1000,Июнь,43,172
4,1000,Май,22,159
5,1000,Ноябрь,43,338
6,1000,Октябрь,57,350
7,1000,Сентябрь,58,466
8,1001,Декабрь,63,414
9,1001,Ноябрь,59,430


Изучим теперь количество отправленных сообщений

In [16]:
messages.head(10)

,id,message_date,user_id
0,1000_0,2018-06-27,1000
1,1000_1,2018-10-08,1000
2,1000_2,2018-08-04,1000
3,1000_3,2018-06-16,1000
4,1000_4,2018-12-05,1000
5,1000_5,2018-06-20,1000
6,1000_6,2018-11-19,1000
7,1000_7,2018-10-29,1000
8,1000_8,2018-06-25,1000
9,1000_9,2018-12-18,1000


Посмотрим, есть ли пропуски

In [17]:
messages.isnull().sum()

id              0
message_date    0
user_id         0
dtype: int64

Пропусков нет. Рассчитаем количество отправленных сообщений, сгруппировав по месяцам

In [18]:
messages['month'] = pd.to_datetime(messages['message_date']).dt.month
messages['month'].replace(months, inplace=True)

Применим сводную таблицу, использовав в качестве `aggfunc` функцию `count`, посчитав все `id` сообщений, получив таким образом их количество 

In [19]:
user_msgs_by_months = (messages
             .pivot_table(index=['user_id', 'month'], values='id', aggfunc=['count'])
             .reset_index())
user_msgs_by_months.columns=['user_id', 'month', 'total_messages'] # меняем название столбцов на более логичные
user_msgs_by_months.head(10)

,user_id,month,total_messages
0,1000,Август,81
1,1000,Декабрь,70
2,1000,Июль,75
3,1000,Июнь,60
4,1000,Май,22
5,1000,Ноябрь,58
6,1000,Октябрь,73
7,1000,Сентябрь,57
8,1002,Август,13
9,1002,Декабрь,12


Посмотрим расходы интернет-траффика

In [20]:
internet.head(10)

,Unnamed: 0,id,mb_used,session_date,user_id
0,0,1000_0,112.95,2018-11-25,1000
1,1,1000_1,1052.81,2018-09-07,1000
2,2,1000_2,1197.26,2018-06-25,1000
3,3,1000_3,550.27,2018-08-22,1000
4,4,1000_4,302.56,2018-09-24,1000
5,5,1000_5,399.97,2018-10-02,1000
6,6,1000_6,540.08,2018-09-07,1000
7,7,1000_7,415.70,2018-07-11,1000
8,8,1000_8,505.40,2018-10-08,1000
9,9,1000_9,345.54,2018-09-03,1000


Видим странный столбец `Unnamed: 0`, в котором просто записаны уникальные id. Нам этот столбец никак не пригодится, поэтому отбросим его

In [21]:
internet.drop(columns='Unnamed: 0')

,id,mb_used,session_date,user_id
0,1000_0,112.95,2018-11-25,1000
1,1000_1,1052.81,2018-09-07,1000
2,1000_2,1197.26,2018-06-25,1000
3,1000_3,550.27,2018-08-22,1000
4,1000_4,302.56,2018-09-24,1000
...,...,...,...,...
149391,1499_152,318.90,2018-10-03,1499
149392,1499_153,490.13,2018-12-14,1499
149393,1499_154,0.00,2018-10-27,1499
149394,1499_155,1246.32,2018-11-26,1499


Сразу округлим значения потраченного трафика в большую сторону и привёдём к целому типу

In [22]:
internet['mb_used'] = np.ceil(internet['mb_used']).astype('int')

Выделим месяц из даты и сгруппируем по месяцам, посчитав расход трафика для каждого пользователя

In [23]:
internet['month'] = pd.to_datetime(internet['session_date']).dt.month
internet['month'].replace(months, inplace=True)

internet_by_months = (internet
             .pivot_table(index=['user_id', 'month'], values='mb_used', aggfunc=['sum'])
             .reset_index())

internet_by_months.columns=['user_id', 'month', 'total_trafic_gb'] # меняем название столбцов на более логичные
internet_by_months['total_trafic_gb'] = internet_by_months['total_trafic_gb'] / 1024
internet_by_months.head(10)

,user_id,month,total_trafic_gb
0,1000,Август,13.740234
1,1000,Декабрь,9.596680
2,1000,Июль,13.687500
3,1000,Июнь,22.711914
4,1000,Май,2.203125
5,1000,Ноябрь,14.421875
6,1000,Октябрь,14.372070
7,1000,Сентябрь,14.239258
8,1001,Декабрь,13.723633
9,1001,Ноябрь,18.019531


Сведём теперь это всё в единую таблицу, сгруппировав по `user_id` и по `month`

In [24]:
tariffs.rename(columns={'tariff_name':'tariff'}, inplace=True)

In [25]:
users_income = users.merge(user_calls_by_months, how='outer')
users_income = users_income.merge(user_msgs_by_months, how='outer', on=['user_id', 'month'])
users_income = users_income.merge(internet_by_months, how='outer', on=['user_id', 'month'])
users_income = users_income.merge(tariffs, how='outer', on=['tariff'])
#users_income.tail(50)


In [26]:
users_income.fillna(0, inplace=True)
users_income

,user_id,age,churn_date,city,first_name,last_name,reg_date,tariff,month,total_calls,total_minutes,total_messages,total_trafic_gb,messages_included,mb_per_month_included,minutes_included,rub_monthly_fee,rub_per_gb,rub_per_message,rub_per_minute
0,1000,52.0,0,Краснодар,Рафаил,Верещагин,2018-05-25,ultra,Август,52.0,408.0,81.0,13.740234,1000.0,30720.0,3000.0,1950.0,150.0,1.0,1.0
1,1000,52.0,0,Краснодар,Рафаил,Верещагин,2018-05-25,ultra,Декабрь,46.0,333.0,70.0,9.596680,1000.0,30720.0,3000.0,1950.0,150.0,1.0,1.0
2,1000,52.0,0,Краснодар,Рафаил,Верещагин,2018-05-25,ultra,Июль,47.0,340.0,75.0,13.687500,1000.0,30720.0,3000.0,1950.0,150.0,1.0,1.0
3,1000,52.0,0,Краснодар,Рафаил,Верещагин,2018-05-25,ultra,Июнь,43.0,172.0,60.0,22.711914,1000.0,30720.0,3000.0,1950.0,150.0,1.0,1.0
4,1000,52.0,0,Краснодар,Рафаил,Верещагин,2018-05-25,ultra,Май,22.0,159.0,22.0,2.203125,1000.0,30720.0,3000.0,1950.0,150.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3217,1489,0.0,0,0,0,0,0,0,Декабрь,0.0,0.0,35.0,16.934570,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3218,1489,0.0,0,0,0,0,0,0,Ноябрь,0.0,0.0,20.0,17.105469,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3219,1489,0.0,0,0,0,0,0,0,Октябрь,0.0,0.0,21.0,19.128906,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3220,1489,0.0,0,0,0,0,0,0,Сентябрь,0.0,0.0,32.0,16.828125,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Видим, что начиная с 3181 строки, почти все столбцы заполнились None. И т.к. тариф не указан, заполним пропуски тарифа исходя из среднего количества расходуемого интернет-трафика, чтобы посчитать прибыль. Если расход трафика больше, чем среднее + стандартное отклонение для тарифа смарт, то мы устанавливаем тариф ультра, иначе тариф смарт

In [27]:
users_smart = users_income.loc[users_income['tariff'] == 'smart']
null_tariff = users_income.query('tariff == 0') # берем срез пользователей без тарифа
smart_mean_trafic = users_smart['total_trafic_gb'].mean() # считаем средний расход пользователей трафика тарифа смарт
smart_std_trafic = np.std(users_smart['total_trafic_gb']) # считаем среднее отклонение

pivot_tariff_internet = null_tariff.pivot_table(index=['user_id'], values='total_trafic_gb', aggfunc='mean').reset_index()
pivot_tariff_internet

,user_id,total_trafic_gb
0,1091,13.122681
1,1106,16.960829
2,1231,27.188843
3,1328,31.730143
4,1373,12.165039
5,1473,16.416211
6,1476,0.518555
7,1489,16.160742


In [28]:
def set_tariff(users):
    internet_trafic = users['total_trafic_gb']
    
    for element in pivot_tariff_internet['user_id']:
        if internet_trafic > (smart_mean_trafic + smart_std_trafic):
            return 'ultra'
        return 'smart'

In [29]:
users_income['tariff'] = users_income.apply(set_tariff, axis=1)
users_income.tail(10)

,user_id,age,churn_date,city,first_name,last_name,reg_date,tariff,month,total_calls,total_minutes,total_messages,total_trafic_gb,messages_included,mb_per_month_included,minutes_included,rub_monthly_fee,rub_per_gb,rub_per_message,rub_per_minute
3212,1473,0.0,0,0,0,0,0,smart,Декабрь,0.0,0.0,20.0,18.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3213,1473,0.0,0,0,0,0,0,smart,Ноябрь,0.0,0.0,14.0,21.183594,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3214,1473,0.0,0,0,0,0,0,smart,Октябрь,0.0,0.0,20.0,13.381836,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3215,1473,0.0,0,0,0,0,0,smart,Сентябрь,0.0,0.0,19.0,16.901367,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3216,1489,0.0,0,0,0,0,0,smart,Август,0.0,0.0,17.0,10.806641,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3217,1489,0.0,0,0,0,0,0,smart,Декабрь,0.0,0.0,35.0,16.934570,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3218,1489,0.0,0,0,0,0,0,smart,Ноябрь,0.0,0.0,20.0,17.105469,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3219,1489,0.0,0,0,0,0,0,smart,Октябрь,0.0,0.0,21.0,19.128906,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3220,1489,0.0,0,0,0,0,0,smart,Сентябрь,0.0,0.0,32.0,16.828125,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3221,1476,0.0,0,0,0,0,0,smart,Апрель,0.0,0.0,0.0,0.518555,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Преобразуем типы данных

In [30]:
to_int = ['age', 'total_calls', 'total_minutes', 'total_messages', 'messages_included', 'mb_per_month_included',
          'minutes_included', 'rub_monthly_fee', 'rub_per_gb', 'rub_per_message', 'rub_per_minute']
for elem in to_int:
    users_income[elem] = users_income[elem].astype('int')

Расчитаем теперь месячную выручку с компании для каждого абонента по звонкам, смс, интернет траффику и суммарную

In [32]:
SMART_PAYMENT = 550 # цена в рублях

SMART_MINUTES = 500  # количество минут
SMART_SMS = 50
SMART_TRAFFIC = 15 # гб

SMART_MINUTES_OVER = 3
SMART_SMS_OVER = 3
SMART_TRAFFIC_OVER = 200

ULTRA_PAYMENT = 1950   #цена в рублях

ULTRA_MINUTES = 3000
ULTRA_SMS = 1000
ULTRA_TRAFFIC = 30  # учитываем трафик в гб

ULTRA_MINUTES_OVER = 1
ULTRA_SMS_OVER = 1
ULTRA_TRAFFIC_OVER = 150

In [33]:
def calc_income(users):
    '''Функция считает прибыль в зависимости от тарифа'''
    income_smart = SMART_PAYMENT # 2 переменных, чтобы не изменять значения констант
    income_ultra = ULTRA_PAYMENT
    
    tariff = users['tariff']
    total_minutes = users['total_minutes']
    total_messages = users['total_messages']
    total_traffic = users['total_trafic_gb']
    
    if tariff == 'smart':
        
        if total_minutes >= SMART_MINUTES:
            income_smart += (SMART_MINUTES_OVER * (total_minutes - SMART_MINUTES))
    
        if total_messages >= SMART_SMS:
            income_smart += (SMART_SMS_OVER * (total_messages - SMART_SMS))
    
        if total_traffic >= SMART_TRAFFIC:
            income_smart += (SMART_TRAFFIC_OVER * (np.ceil(total_traffic - SMART_TRAFFIC)))# округляем мб вверх и переводим в гб
    
        return income_smart
    
    if tariff == 'ultra':
        
        if total_minutes >= ULTRA_MINUTES:
            income_ultra += (ULTRA_MINUTES_OVER * (total_minutes - ULTRA_MINUTES))
    
        if total_messages >= ULTRA_SMS:
            income_ultra += (ULTRA_SMS_OVER * (total_messages - ULTRA_SMS))
    
        if total_traffic >= ULTRA_TRAFFIC:
            income_ultra += (ULTRA_TRAFFIC_OVER * (np.ceil(total_traffic - ULTRA_TRAFFIC)))
    
        return income_ultra

Применим теперь функцию `calc_income` к нашему датасету

In [34]:
users_income['income'] = users_income.apply(calc_income, axis=1)
users_income.head(10)
#users_income = users_income.loc[users_income['month'] != 0]

,user_id,age,churn_date,city,first_name,last_name,reg_date,tariff,month,total_calls,...,total_messages,total_trafic_gb,messages_included,mb_per_month_included,minutes_included,rub_monthly_fee,rub_per_gb,rub_per_message,rub_per_minute,income
0,1000,52,0,Краснодар,Рафаил,Верещагин,2018-05-25,smart,Август,52,...,81,13.740234,1000,30720,3000,1950,150,1,1,643.0
1,1000,52,0,Краснодар,Рафаил,Верещагин,2018-05-25,smart,Декабрь,46,...,70,9.596680,1000,30720,3000,1950,150,1,1,610.0
2,1000,52,0,Краснодар,Рафаил,Верещагин,2018-05-25,smart,Июль,47,...,75,13.687500,1000,30720,3000,1950,150,1,1,625.0
3,1000,52,0,Краснодар,Рафаил,Верещагин,2018-05-25,ultra,Июнь,43,...,60,22.711914,1000,30720,3000,1950,150,1,1,1950.0
4,1000,52,0,Краснодар,Рафаил,Верещагин,2018-05-25,smart,Май,22,...,22,2.203125,1000,30720,3000,1950,150,1,1,550.0
5,1000,52,0,Краснодар,Рафаил,Верещагин,2018-05-25,smart,Ноябрь,43,...,58,14.421875,1000,30720,3000,1950,150,1,1,574.0
6,1000,52,0,Краснодар,Рафаил,Верещагин,2018-05-25,smart,Октябрь,57,...,73,14.372070,1000,30720,3000,1950,150,1,1,619.0
7,1000,52,0,Краснодар,Рафаил,Верещагин,2018-05-25,smart,Сентябрь,58,...,57,14.239258,1000,30720,3000,1950,150,1,1,571.0
8,1003,23,0,Москва,Белла,Белякова,2018-08-17,smart,Август,55,...,37,8.370117,1000,30720,3000,1950,150,1,1,550.0
9,1003,23,0,Москва,Белла,Белякова,2018-08-17,smart,Декабрь,108,...,75,9.894531,1000,30720,3000,1950,150,1,1,1531.0


### Анализ данных

Рассмотрим поведение пользователей в зависимости от тарифа

In [35]:
users_ultra = users_income.query('tariff == "ultra"')
users_smart = users_income.query('tariff == "smart"')

Распределение по звонкам

In [51]:
fig = px.histogram(users_income, x='total_minutes', color='tariff', title='Количество потраченных минут')
fig.show()

В целом, распределение достаточно близко к нормальному. Единственное, что выпадает - это нулевые значения, т.е. пользователи, которые не пользовались звонками.

Так же мы видим, что пользователи тарифа ультра тратят больше минут, чем пользователи тарифа смарт

Посмотрим основные статистические величины для звонков

In [37]:
smart_mean_minutes = users_smart['total_minutes'].mean()
smart_variance_minutes = np.var(users_smart['total_minutes'])
smart_std_minutes = np.std(users_smart['total_minutes'])

ultra_mean_minutes = users_ultra['total_minutes'].mean()
ultra_variance_minutes = np.var(users_ultra['total_minutes'])
ultra_std_minutes = np.std(users_ultra['total_minutes'])

print('Среднее количество минут для тарифа смарт:', smart_mean_minutes)
print('Дисперсия по количеству минут для тарифа смарт:', smart_variance_minutes)
print('Стандартное отклонение количества минут для тарифа смарт:', smart_std_minutes)
print()
print('Среднее количество минут для тарифа ультра:', ultra_mean_minutes)
print('Дисперсия по количеству минут для тарифа ультра:', ultra_variance_minutes)
print('Стандартное отклонение количества минут для тарифа ультра:', ultra_std_minutes)

Среднее количество минут для тарифа смарт: 430.7285996055227
Дисперсия по количеству минут для тарифа смарт: 51599.557111056645
Стандартное отклонение количества минут для тарифа смарт: 227.15535897499018

Среднее количество минут для тарифа ультра: 521.6943231441048
Дисперсия по количеству минут для тарифа ультра: 79146.39273342105
Стандартное отклонение количества минут для тарифа ультра: 281.3296869038549


Среднее количество минут у пользователей разных тарифов отличается почти на 100

Посмотрим распределение по сообщениям

In [38]:
fig = px.histogram(users_income, x='total_messages', color='tariff', title='Количество отправленных сообщений')
fig.show()

Значение в нуле выделяется более чем в несколько раз над остальными. Это говорит нам о том, что смс мало востребованы. Что и логично в эпоху мессенджеров и социальных сетей

Посмотрим основные статистические величины для звонков

In [39]:
smart_mean_messages = users_smart['total_messages'].mean()
smart_variance_messages = np.var(users_smart['total_messages'])
smart_std_messages = np.std(users_smart['total_messages'])

ultra_mean_messages = users_ultra['total_messages'].mean()
ultra_variance_messages = np.var(users_ultra['total_messages'])
ultra_std_messages = np.std(users_ultra['total_messages'])

print('Среднее количество смс для тарифа смарт:', smart_mean_messages)
print('Дисперсия по количеству смс для тарифа смарт:', smart_variance_messages)
print('Стандартное отклонение количества смс для тарифа смарт:', smart_std_messages)
print()
print('Среднее количество смс для тарифа ультра:', ultra_mean_messages)
print('Дисперсия по количеству смс для тарифа ультра:', ultra_variance_messages)
print('Стандартное отклонение количества смс для тарифа ультра:', ultra_std_messages)

Среднее количество смс для тарифа смарт: 35.791321499013804
Дисперсия по количеству смс для тарифа смарт: 1068.1477747822398
Стандартное отклонение количества смс для тарифа смарт: 32.68253011598459

Среднее количество смс для тарифа ультра: 47.02328966521106
Дисперсия по количеству смс для тарифа ультра: 2087.600622074755
Стандартное отклонение количества смс для тарифа ультра: 45.69026835196698


И хотя среднее значение в 35 сообщений для пользователей тарифа смарт и в 47 сообщений для пользователей тарифа ультра кажется большим, то взглянув на стандартное отклонение, которое достаточно близко к среднему, мы сразу понимаем, что есть небольшое количество пользователей, которые используют услугу смс-сообщений очень активно и из-за этого "перетягивают" среднее в свою сторону. Но стандартное отклонение, близкое к среднему - выдает нам, что это среднее очень условное

Посмотрим распределение по интернет-траффику

In [40]:
fig = px.histogram(users_income, x='total_trafic_gb', color='tariff', title='Количество потраченного интернет-траффика')
fig.show()

In [41]:
users_ultra['total_trafic_gb'].min()

21.6064453125

Получили очень интересное распределение, в котором все пользователи тарифа смарт используют до 22.5 гб интернет-траффика, а пользовватели тарифа ультра используют от 21.5 гб интернет-траффика. Видимо, пользователи тарифа ультра стараются выжать максимум интернета из своих услуг и используют его очень активно

Посмотрим основные статистические величины для интернет-трафика

In [42]:
smart_mean_trafic = users_smart['total_trafic_gb'].mean()
smart_variance_trafic = np.var(users_smart['total_trafic_gb'])
smart_std_trafic = np.std(users_smart['total_trafic_gb'])

ultra_mean_trafic = users_ultra['total_trafic_gb'].mean()
ultra_variance_trafic = np.var(users_ultra['total_trafic_gb'])
ultra_std_trafic = np.std(users_ultra['total_trafic_gb'])

print('Среднее количество интернет-трафика для тарифа смарт:', smart_mean_trafic)
print('Дисперсия по количеству интернет-трафика для тарифа смарт:', smart_variance_trafic)
print('Стандартное отклонение количества интернет-трафика для тарифа смарт:', smart_std_trafic)
print()
print('Среднее количество интернет-трафика для тарифа ультра:', ultra_mean_trafic)
print('Дисперсия по количеству интернет-трафика для тарифа ультра:', ultra_variance_trafic)
print('Стандартное отклонение количества интернет-трафика для тарифа ультра:', ultra_std_trafic)

Среднее количество интернет-трафика для тарифа смарт: 13.987206068170611
Дисперсия по количеству интернет-трафика для тарифа смарт: 26.68770538275436
Стандартное отклонение количества интернет-трафика для тарифа смарт: 5.166014458240933

Среднее количество интернет-трафика для тарифа ультра: 27.09486160389374
Дисперсия по количеству интернет-трафика для тарифа ультра: 25.783760464851756
Стандартное отклонение количества интернет-трафика для тарифа ультра: 5.07777121036895


Интересно, что средний расход интернет-траффика у разных тарифов отличается почти в 2 раза, а вот стандартное отклонение можно считать равынм

### Формулировка и проверка гипотез

**Гипотеза 1**. Средняя выручка пользователей тарифов «Ультра» и «Смарт» **не** различается - это и будет нашей нулевой гипотезой.

Средняя выручка пользователей тарифов «Ультра» и «Смарт» различается - будет альтернативной гипотезой

Воспользуемся t-критерием о равенстве средних двух генеральных совокупностей

In [43]:
alpha = 0.05 # выбирем стандартный уровень значимости

results = st.ttest_ind(
    users_smart['income'],
    users_ultra['income'], equal_var=False)

print('p-значение:', results.pvalue)

if (results.pvalue < alpha):
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу")

p-значение: 1.1226642771715188e-307
Отвергаем нулевую гипотезу


То есть различие по средней выручке всё-таки есть. Но по t-критерию мы не можем определить, у кого она больше или меньше

Перед проверкой второй гипотезы, разобьем пользователей на 2 группы. Тех, кто из Москвы и тех, кто не из Москвы. И проверим отдельно для каждого из тарифов

In [44]:
def is_moscow(users):
    return True if users['city'] == 'Москва' else False

In [45]:
users_smart['is_moscow'] = users_smart.apply(is_moscow, axis=1)
users_ultra['is_moscow'] = users_ultra.apply(is_moscow, axis=1)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



**Гипотеза 2**. Средняя выручка пользователей из Москвы **не** отличается от выручки пользователей из других регионов - нулевая гипотеза

Средняя выручка пользователей из Москвы отличается от выручки пользователей из других регионов - альтернативная гипотеза

Воспользуемся t-критерием о равенстве средних двух генеральных совокупностей

Для тарифа смарт:

In [46]:
users_smart_moscow = users_smart.loc[users_smart['is_moscow']]
users_smart_region = users_smart.loc[~users_smart['is_moscow']]

In [47]:
alpha = 0.05 # выбирем стандартный уровень значимости

results = st.ttest_ind(
    users_smart_moscow['income'],
    users_smart_region['income'], equal_var=False)

print('p-значение:', results.pvalue)

if (results.pvalue < alpha):
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу")

p-значение: 0.20126044217933434
Не получилось отвергнуть нулевую гипотезу


Итак, отвергнуть нулевую гипотезу не получилось. Это означает, что для пользователи тарифа смарт дают компании примерно одинаковую выручку как в Москве, так и в регионах

Посмотрем теперь то же, но для пользователей тарифа ультра

In [48]:
users_ultra_moscow = users_ultra.loc[users_ultra['is_moscow']]
users_ultra_region = users_ultra.loc[~users_ultra['is_moscow']]

In [49]:
alpha = 0.05 # выбирем стандартный уровень значимости

results = st.ttest_ind(
    users_ultra_moscow['income'],
    users_ultra_region['income'], equal_var=False)

print('p-значение:', results.pvalue)

if (results.pvalue < alpha):
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу")

p-значение: 0.9337222657191679
Не получилось отвергнуть нулевую гипотезу


Как и для пользователей тарифа смарт, нулевую гипотезу отвергнуть не удалось. То есть регион не влияет на выручку с абонента

### Вывод

Сразу стоит отметить, что анализ проведен на выборке из 500 пользователей, а не на общей совокупности. Это означает, что есть маленькая доля вероятности того, что поведение пользователей из генеральной совокупности может несколько отличаться от поведения пользователей в этой выборке

Пользователи тарифа ультра в среднем не используют даже и половины из всех минут или смс. А вот интернет-трафика они используют больше 15гб, в среднем - 19гб. Скорее всего, люди и выбирают этот тариф, чтобы не переплачивать за интернет за каждый гб, а иметь в запасе сразу достаточное его количество.

В целом, смс и звонки почти неактуальны. Что весьма логично, потому что используя интернет, можно и писать сообщения в социальных сетях и звонить через мессенджеры

Первая гипотеза о том, что выручка с пользователей разных тарифов одинакова - была опровергнута. Больше выручка с пользователей тарифа ультра, потому что в нем очень высокая базовая стоимость. И даже при выходах за лимиты у пользователей тарифа смарт - это не дает такой прибыли, как базовая стоимость тарифа ультра

Вторая гипотеза о том, что выручка в Москве и в регионах примерно одинакова - опровергнуть не получилось. Ни для тарифа ультра, ни для тарифа смарт.

Соответственно, для того чтобы увеличить выручку компании - надо больше продвигать тариф ультра